In [2]:
from keras.models import load_model
import cv2
import numpy as np

Using TensorFlow backend.


# Load the Model and Cascade Classifier

In [3]:
## https://stackoverflow.com/questions/30508922/error-215-empty-in-function-detectmultiscale
model = load_model('Model_12.model')
face_clsfr = cv2.CascadeClassifier(r'C:\Users\KIIT\Tensorflow\Face_mask_detection\cascade\haarcascade_frontalface_default.xml')
source = cv2.VideoCapture(r'C:\Users\KIIT\Tensorflow\Face_mask_detection\Demo\mask.mp4')

labels_dict = {0 : 'Mask', 1 : 'No Mask'}
color_dict = {0 : (0,255,0), 1 : (0, 0, 255)}

In [4]:
while(True):

    ret, img = source.read()
    grayMode_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_clsfr.detectMultiScale(grayMode_img) 

    for (x, y, w, h) in faces:
        face_img = grayMode_img[y:y+h, x:x+w]              ## Extract Box which contains face  
        resized_img = cv2.resize(face_img, (100, 100))     ## Resize to (100 X 100 Dimmension)
        normalized = resized_img / (255.0)                 ## Normalize 100 X 100 Image
        reshaped = np.reshape(normalized, (1, 100, 100, 1))## Reshape to (1 X 100 X 100 X 1) Dimmension, for the model
        res = model.predict(reshaped)                      ## Model Prediction
        label = np.argmax(res, axis=1)[0]                  ## Extract the Label from the model Prediction

        cv2.rectangle(img,(x, y),(x + w, y + h), color_dict[label], 2)
        cv2.rectangle(img,(x, y - 40),(x + w, y), color_dict[label], -1)
        cv2.putText(img, labels_dict[label], (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
        
        ## Accuracy
        accuracy = round(np.max(res, axis=1)[0]*100, 2)
        cv2.putText(img, str(accuracy),(x + 150, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255,255,255), 2)
            
    cv2.imshow('LIVE',img)
    key = cv2.waitKey(1)
    
    ## Press Esc to Destroy the Window
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()